In [ ]:
// Pastikan 'lembang2' sudah didefinisikan sebagai ee.FeatureCollection atau ee.Geometry
// Contoh:
// var lembang2 = ee.FeatureCollection('users/YOUR_USERNAME/YOUR_ASSET_NAME');
// Map.centerObject(lembang2, 10);
// Map.addLayer(lembang2, {}, 'Lembang Boundary');

// --- Fungsi Pembantu ---

// Cloud Masking Function for Landsat 8
var masking = function(img) {
  var cloudshadowBitmask = (1 << 4);
  var cloudshadowmask = (1 << 3);
  var qa = img.select('QA_PIXEL');
  var maskshadow = qa.bitwiseAnd(cloudshadowBitmask).eq(0);
  var maskcloud = qa.bitwiseAnd(cloudshadowmask).eq(0);
  var mask = maskshadow.and(maskcloud);
  return img.updateMask(mask)
            .copyProperties(img, ['system:time_start', 'system:index', 'CLOUD_COVER_LAND']);
};

// Scaling Factor Function for Landsat 8 Thermal Band
var scaleLST = function applyScaleFactors(image) {
  // Hanya ambil band thermal dan terapkan scaling
  var thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0);
  return image.addBands(thermalBand.rename('LST_Kelvin'), null, true); // Rename untuk kejelasan
};

// --- Definisi Dataset ---
var Landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");

// --- Rentang Waktu dan Parameter ---
var startYear = 2013;
var startMonth = 5; // Mei 2013, karena data publik Landsat 8 dimulai sekitar 30 Mei 2013
var endYear = 2022;
var endMonth = 12; // Desember
var numSamples = 1000; // Jumlah titik sampel acak per bulan
var scaleForSampling = 30; // Resolusi asli LST Landsat (m) untuk sampling

// --- Fungsi untuk Memproses Data LST Bulanan ---
var processMonthlyLST = function(date) {
  date = ee.Date(date);
  var endDate = date.advance(1, 'month');

  // Filter Landsat 8 untuk bulan ini
  var L8_monthly_raw = Landsat_8.filterBounds(lembang2)
                                .filterDate(date, endDate);

  // Debugging: Hitung berapa citra Landsat yang ada sebelum masking
  // print('Raw L8 images for ' + date.format('YYYY-MM') + ':', L8_monthly_raw.size());

  // Apply masking and scaling for LST
  var L8_monthly_processed = L8_monthly_raw
                            .map(masking)
                            .map(scaleLST);

  // Debugging: Hitung berapa citra Landsat yang ada setelah masking
  // print('Processed L8 images for ' + date.format('YYYY-MM') + ':', L8_monthly_processed.size());

  // Hitung median (komposit) untuk bulan tersebut.
  var LST_Image_Kelvin_Collection = L8_monthly_processed.select('LST_Kelvin');

  var LST_Image_Kelvin_Median = ee.Image(LST_Image_Kelvin_Collection.median()).clip(lembang2);

  // **Perbaikan inti:** Periksa apakah citra median memiliki band `'LST_Kelvin'`.
  var hasLandsatData = LST_Image_Kelvin_Median.bandNames().contains('LST_Kelvin');

  if (!hasLandsatData) {
    // Debugging: uncomment baris di bawah untuk melihat bulan mana yang kosong
    // print('No valid LST data for ' + date.format('YYYY-MM'));
    return ee.FeatureCollection([]); // Kembalikan koleksi kosong jika tidak ada data
  }

  // Konversi LST dari Kelvin ke Celcius
  var LST_celcius = LST_Image_Kelvin_Median.subtract(273.15).rename('LST_C');

  // --- Sampling Acak ---
  var samplePoints = LST_celcius.sample({
    region: lembang2,
    scale: scaleForSampling,
    numPixels: numSamples,
    geometries: true,
    seed: 0,
    tileScale: 16
  });

  // Tambahkan properti waktu ke setiap fitur sampel
  samplePoints = samplePoints.map(function(feature) {
    return feature.set({
      'year': date.get('year'),
      'month': date.get('month'),
      'date': date.format('YYYY-MM-dd')
    });
  });

  // Debugging: uncomment baris di bawah untuk melihat jumlah sampel per bulan
  // print('Samples for ' + date.format('YYYY-MM') + ':', samplePoints.size());

  return samplePoints;
};

// --- Buat daftar semua bulan dalam rentang waktu ---
var dates = [];
for (var y = startYear; y <= endYear; y++) {
  var currentMonth = (y === startYear) ? startMonth : 1; // Mulai dari Mei untuk tahun pertama, Januari untuk tahun berikutnya
  var lastMonth = (y === endYear) ? endMonth : 12; // Akhiri di Desember untuk tahun terakhir, Desember untuk tahun sebelumnya

  for (var m = currentMonth; m <= lastMonth; m++) {
    dates.push(ee.Date.fromYMD(y, m, 1));
  }
}

// Proses data untuk setiap bulan
var all_sampled_LST_data = ee.FeatureCollection(ee.List(dates).map(processMonthlyLST)).flatten();

// --- Ekspor Data ke Google Drive ---
Export.table.toDrive({
  collection: all_sampled_LST_data,
  description: 'LST_Monthly_Lembang_Samples_From_L8_Start',
  folder: 'GEE_Kekeringan_Data',
  fileFormat: 'CSV'
});

// --- Opsional: Visualisasi satu contoh bulan (Misal, Juli 2013) ---
/*
var testDate = ee.Date.fromYMD(2013, 7, 1); // Coba bulan Juli 2013
var L8_monthly_test_raw = Landsat_8.filterBounds(lembang2).filterDate(testDate, testDate.advance(1, 'month'));
var L8_monthly_test_processed = L8_monthly_test_raw.map(masking).map(scaleLST);
var LST_Image_Kelvin_Median_test = ee.Image(L8_monthly_test_processed.select('LST_Kelvin').median()).clip(lembang2);
var LST_celcius_test_image = LST_Image_Kelvin_Median_test.subtract(273.15).rename('LST_C');

var LSTParams = {min: 20, max: 40, palette: ['blue', 'cyan', 'green', 'yellow', 'red']};
Map.addLayer(LST_celcius_test_image, LSTParams, 'LST Test Image ' + testDate.format('YYYY-MM'));
Map.addLayer(lembang2, {}, 'Lembang Boundary');
Map.centerObject(lembang2, 10);
print('Example LST image for ' + testDate.format('YYYY-MM') + ':', LST_celcius_test_image);
*/

// Print total sampel yang dihasilkan di konsol
print('Total samples for all months (LST only):', all_sampled_LST_data.size());

/*